In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
uniset = pd.read_csv("Universal set of tags.csv")
kaz2uni = {i[0]: i[1] for i in uniset[['Kazym FW', 'UniMorph']].dropna().values}
wes2uni = {i[0]: i[1] for i in uniset[['Western FW', 'UniMorph']].dropna().values}
web2uni = {i[0]: i[1] for i in uniset[['website', 'UniMorph']].dropna().values}

In [3]:
def unify(file, dct):
    with open(file, 'r', encoding='utf-8') as f:
        rows = []
        for row in f.read().split('\n'):
            tags = row.split('\t')[2].split(';')

            unitags = [dct[tag] for tag in tags if tag]
            while '-' in unitags:
                unitags.remove('-')

            if ('V.PTCP;PST' in unitags or 'V.PTCP;PRS' in unitags or 'V.CVB' in unitags) and 'V' in unitags:
                unitags.remove('V')

            poss = np.vectorize(lambda x: x[:3] == 'PSS')
            if 'V' in unitags and any(poss(np.array(unitags))):
                wrong = unitags[poss(np.array(unitags)).argmax()]
                unitags[poss(np.array(unitags)).argmax()] = f'ARGNO{wrong[-2]};ARGNO{wrong[-1]}'

            if 'V' in unitags and 'PL' in unitags:
                unitags.remove('PL')
                unitags.append('ARGNO3;ARGNOP')
            
            arg = np.vectorize(lambda x: x[:3] == 'ARG')
            if 'V' in unitags and not any(arg(np.array(unitags))):
                unitags.append('ARGNO3;ARGNOS')
            
            if 'V.PTCP;PST' in unitags and any(poss(np.array(unitags))):
                unitags.remove('V.PTCP;PST')
                unitags.insert(0, 'V;PST')
                wrong = unitags[poss(np.array(unitags)).argmax()]
                unitags[poss(np.array(unitags)).argmax()] = f'INFER;ARGNO{wrong[-2]};ARGNO{wrong[-1]}'
            
            if 'V.PTCP;PRS' in unitags and any(poss(np.array(unitags))):
                unitags.remove('V.PTCP;PRS')
                unitags.insert(0, 'V;PRS')
                wrong = unitags[poss(np.array(unitags)).argmax()]
                unitags[poss(np.array(unitags)).argmax()] = f'INFER;ARGNO{wrong[-2]};ARGNO{wrong[-1]}'
            
            if 'N' in unitags and 'ADJ' in unitags:
                unitags.remove('N')
            
            if 'N' in unitags and any(arg(np.array(unitags))):
                wrong = unitags[arg(np.array(unitags)).argmax()]
                unitags[arg(np.array(unitags)).argmax()] = f'PSS{wrong[5]}{wrong[12]}'

            if ('N' in unitags or 'PRO' in unitags) and ('DAT' not in unitags and 'LOC' not in unitags and 'ACC' not in unitags):
                unitags.append('NOM')

            if ('N' in unitags or 'PRO' in unitags) and 'PL' not in unitags:
                unitags.append('SG')

            if 'NUM' in unitags and 'ADJ' in unitags:
                unitags.remove('NUM')
            
            if 'ADV' in unitags and 'ADJ' in unitags:
                unitags.remove('ADV')

            tagstring = ';'.join(unitags)
            rows.append(row.split("\t")[0]+'\t'+row.split("\t")[1]+'\t'+tagstring)
    return rows

In [4]:
Unified = unify('Kaz_fw.xml_unidraft.tsv', kaz2uni)
Unified.extend(unify('Wes_fw.xml_unidraft.tsv', wes2uni))
Unified.extend(unify('website_unidraft.tsv', web2uni))

In [5]:
with open('Unified.tsv', 'w', encoding='utf-8') as f:
    f.write('\n'.join(sorted(set(Unified))))